In [2]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter, detrend
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from scipy import stats
import joblib
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv("S_Spectra_A.csv")

# Separate the ID column
ids = data.iloc[:, 0]
data = data.iloc[:, 1:]

# Handle missing values
data.fillna(data.median(), inplace=True)

# Detect and handle outliers using Z-score threshold = 2.8
z_scores_properties = np.abs(stats.zscore(data.iloc[:, :4]))  # Soil properties (Cu, Zn, Fe, Mn)
outliers_properties = (z_scores_properties > 2.8).any(axis=1)
data_no_outliers_properties = data[~outliers_properties]

z_scores_spectra = np.abs(stats.zscore(data_no_outliers_properties.iloc[:, 4:]))  # Spectra data
outliers_spectra = (z_scores_spectra > 2.8).any(axis=1)
data_no_outliers = data_no_outliers_properties[~outliers_spectra]

print("Number of samples after removing outliers:", data_no_outliers.shape[0])

# Separate features (spectra) and targets (soil properties)
X = data_no_outliers.iloc[:, 4:]  # Spectra data
y = data_no_outliers.iloc[:, :4]  # Soil properties (Cu, Zn, Fe, Mn)

# Apply Savitzky-Golay filter for smoothing
X_smoothed = savgol_filter(X, window_length=11, polyorder=2, axis=1)

# Apply baseline correction
X_corrected = detrend(X_smoothed, axis=1)

# Define the preprocessing pipeline with PCA = 0.99
preprocess_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.99))
])

# Fit and transform the data
X_preprocessed = preprocess_pipeline.fit_transform(X_corrected)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

print('Training set shape:', X_train.shape, y_train.shape)
print('Test set shape:', X_test.shape, y_test.shape)

# Define the base models
ridge = Ridge(alpha=507.4417747114648)  # Use the best alpha from previous tuning
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
gbr = GradientBoostingRegressor(n_estimators=50, max_depth=3, learning_rate=0.1)

# Define the stacking ensemble
stacking_model = StackingRegressor(
    estimators=[
        ('ridge', ridge),
        ('rf', random_forest),
        ('gbr', gbr)
    ],
    final_estimator=Ridge()  # Meta-model
)

# Wrap the stacking model with MultiOutputRegressor
multi_output_stacking_model = MultiOutputRegressor(stacking_model)

# Fit the model
multi_output_stacking_model.fit(X_train, y_train)

# Evaluate on the test set for each soil property
properties = ['Cu', 'Zn', 'Fe', 'Mn']
y_pred = multi_output_stacking_model.predict(X_test)

for i, property in enumerate(properties):
    r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
    print(f"R^2 score for {property}: {r2:.4f}")

# Cross-validation on training set
cv_scores = cross_val_score(multi_output_stacking_model, X_train, y_train, cv=10, scoring='r2')
print("Cross-validated R^2 scores:", cv_scores)
print("Mean cross-validated R^2 score:", np.mean(cv_scores))

# Evaluate on the test set
stacking_training_score = multi_output_stacking_model.score(X_train, y_train)
stacking_test_score = multi_output_stacking_model.score(X_test, y_test)
print("Overall Stacking Training R^2 score:", stacking_training_score)
print("Overall Stacking Test R^2 score:", stacking_test_score)

# Save the model
joblib.dump(multi_output_stacking_model, 'multi_output_stacking_model.pkl')

# Save the preprocessing pipeline
joblib.dump(preprocess_pipeline, 'preprocess_pipeline.pkl')

print("Model and preprocessing pipeline saved.")


Number of samples after removing outliers: 309
Training set shape: (247, 39) (247, 4)
Test set shape: (62, 39) (62, 4)
R^2 score for Cu: 0.7357
R^2 score for Zn: 0.5272
R^2 score for Fe: 0.3716
R^2 score for Mn: 0.5697
Cross-validated R^2 scores: [0.40908815 0.35252594 0.64382767 0.50034152 0.56303685 0.67445621
 0.51487616 0.45685194 0.54889991 0.4995486 ]
Mean cross-validated R^2 score: 0.516345295484687
Overall Stacking Training R^2 score: 0.7063043605517193
Overall Stacking Test R^2 score: 0.5510602988481184
Model and preprocessing pipeline saved.


In [1]:
import pandas as pd
import numpy as np
from scipy.signal import savgol_filter, detrend
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from scipy import stats
import joblib

# Load data
data = pd.read_csv("C:/Users/artir/Desktop/05 Aug  Spectral Analysis Model training/S_Spectra_A.csv")

# Separate the ID column
ids = data.iloc[:, 0]
data = data.iloc[:, 1:]

# Handle missing values
data.fillna(data.median(), inplace=True)

# Detect and handle outliers using Z-score threshold = 2.8
z_scores_properties = np.abs(stats.zscore(data.iloc[:, :4]))  # Soil properties (Cu, Zn, Fe, Mn)
outliers_properties = (z_scores_properties > 2.8).any(axis=1)
data_no_outliers_properties = data[~outliers_properties]

z_scores_spectra = np.abs(stats.zscore(data_no_outliers_properties.iloc[:, 4:]))  # Spectra data
outliers_spectra = (z_scores_spectra > 2.8).any(axis=1)
data_no_outliers = data_no_outliers_properties[~outliers_spectra]

print("Number of samples after removing outliers:", data_no_outliers.shape[0])

# Separate features (spectra) and targets (soil properties)
X = data_no_outliers.iloc[:, 4:]  # Spectra data
y = data_no_outliers.iloc[:, :4]  # Soil properties (Cu, Zn, Fe, Mn)

# Apply Savitzky-Golay filter for smoothing
X_smoothed = savgol_filter(X, window_length=11, polyorder=2, axis=1)

# Apply baseline correction
X_corrected = detrend(X_smoothed, axis=1)

# Define the preprocessing pipeline with PCA = 0.99
preprocess_pipeline = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.99))
])

# Fit and transform the data
X_preprocessed = preprocess_pipeline.fit_transform(X_corrected)

X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

print('Training set shape:', X_train.shape, y_train.shape)
print('Test set shape:', X_test.shape, y_test.shape)

# Define the base models
ridge = Ridge(alpha=507.4417747114648)  # Use the best alpha from previous tuning
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
gbr = GradientBoostingRegressor(n_estimators=50, max_depth=3, learning_rate=0.1)

# Define the stacking ensemble
stacking_model = StackingRegressor(
    estimators=[
        ('ridge', ridge),
        ('rf', random_forest),
        ('gbr', gbr)
    ],
    final_estimator=Ridge()  # Meta-model
)

# Wrap the stacking model with MultiOutputRegressor
multi_output_stacking_model = MultiOutputRegressor(stacking_model)

# Fit the model
multi_output_stacking_model.fit(X_train, y_train)

# Evaluate training and testing scores for each property individually
y_train_pred = multi_output_stacking_model.predict(X_train)
y_test_pred = multi_output_stacking_model.predict(X_test)

for i, property_name in enumerate(['Cu', 'Zn', 'Fe', 'Mn']):
    train_score = multi_output_stacking_model.estimators_[i].score(X_train, y_train.iloc[:, i])
    test_score = multi_output_stacking_model.estimators_[i].score(X_test, y_test.iloc[:, i])
    print(f"Training R^2 score for {property_name}: {train_score}")
    print(f"Test R^2 score for {property_name}: {test_score}")

# Overall R^2 scores for training and test sets
overall_train_r2 = multi_output_stacking_model.score(X_train, y_train)
overall_test_r2 = multi_output_stacking_model.score(X_test, y_test)

print(f"Overall Stacking Training R^2 score: {overall_train_r2}")
print(f"Overall Stacking Test R^2 score: {overall_test_r2}")

# Cross-validation on training set
cv_scores = cross_val_score(multi_output_stacking_model, X_train, y_train, cv=10, scoring='r2')
print("Cross-validated R^2 scores:", cv_scores)
print("Mean cross-validated R^2 score:", np.mean(cv_scores))

# Save the model and preprocessing pipeline
joblib.dump(multi_output_stacking_model, 'multi_output_stacking_model.pkl')
joblib.dump(preprocess_pipeline, 'preprocess_pipeline.pkl')

print("Model and preprocessing pipeline saved.")


Number of samples after removing outliers: 309
Training set shape: (247, 39) (247, 4)
Test set shape: (62, 39) (62, 4)
Training R^2 score for Cu: 0.8962242410819476
Test R^2 score for Cu: 0.7354994795079735
Training R^2 score for Zn: 0.8619893117210145
Test R^2 score for Zn: 0.5301729319366337
Training R^2 score for Fe: 0.5519511474568712
Test R^2 score for Fe: 0.3624207876866551
Training R^2 score for Mn: 0.519039030809784
Test R^2 score for Mn: 0.5689342087085087
Overall Stacking Training R^2 score: 0.7073009327674044
Overall Stacking Test R^2 score: 0.5492568519599428
Cross-validated R^2 scores: [0.41061913 0.35422395 0.65578944 0.50104736 0.5659061  0.67627937
 0.51723523 0.45615033 0.54908507 0.49408308]
Mean cross-validated R^2 score: 0.5180419058646604
Model and preprocessing pipeline saved.
